In [1]:
# Importing stock ml libraries
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
from data_preprocessing import ANOMALY_LABELS, ANOMALY_SUBCATEGORIES, load_data
from custom_dataset import CustomDataset
from model import SequenceClassificationModel
from metrics import custom_classification_report
from losses import loss

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Defining some key variables that will be used later on in the training
MODEL_NAME = None
MODEL_DIRECTORY = "model_save"


ABBREVIATION = True
# ABBREVIATION = False


LOSS_TYPE = 'BCE'
# LOSS_TYPE = 'BinaryFocal'

BALANCED = False
# BALANCED = True

# LAYERS_TO_UNFREEZE = None
LAYERS_TO_UNFREEZE = [8, 9, 10, 11]

# ENCODER_NAME = 'bert-base-uncased'
# ENCODER_NAME = 'NASA-AIML/MIKA_SafeAeroBERT'
ENCODER_NAME = 'allenai/longformer-base-4096'

IS_SUBCATEGORY = True

if ENCODER_NAME == 'allenai/longformer-base-4096':
    MAX_LEN = 1024
else:
    MAX_LEN = 512

TRAIN_EFFECTIVE_BATCH_SIZE = 32 # 32 Effective size for NASA
TRAIN_BATCH_SIZE = 32
ACCUMULATION_STEPS = TRAIN_EFFECTIVE_BATCH_SIZE / TRAIN_BATCH_SIZE
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA

drop the NaN values in Anomaly?

In [6]:
if IS_SUBCATEGORY:
    labels = ANOMALY_SUBCATEGORIES
else:
    labels = ANOMALY_LABELS
    
if ABBREVIATION:
    train_df = load_data("./data/train_data_final.pkl", labels, pp_path="./data/train_data_processed2.pkl")
    test_df = load_data("./data/test_data_final.pkl", labels, pp_path="./data/test_data_processed2.pkl")
else: 
    train_df = load_data("./data/train_data_final.pkl", labels)
    test_df = load_data("./data/test_data_final.pkl", labels)
train_df

ACN
1163382    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
893734     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
991883     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
1590076    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1715282    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                 ...                        
622204     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
622205     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
661202     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
733747     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
874642     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: labels, Length: 96986, dtype: object

In [7]:
test_df

,text,labels
ACN,,
1014798,Flying SLC DELTA THREE Area Navigation arrival...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1806744,ORD busy east flow arrival push . The weather ...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1044902,B737-800 vectored Instrument Landing System Ru...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1764093,We 6 mile final tower cleared Cirrus land fron...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
1786435,During Climb Leveled 17 ; 000 departure switch...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
1310569,FO flying visual approach runway 26 ZZZ . Wind...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1482118,While assembling GE C2 transfer gearbox ; I no...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1565471,Nearing end hot ; bumpy four-hour Instrument F...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
num_labels = len(test_df.labels.iloc[0])
model = SequenceClassificationModel(ENCODER_NAME, num_labels=num_labels)
model.set_trainable_layers(LAYERS_TO_UNFREEZE)
print(model.to(device))

if ABBREVIATION:
    model.model_name += '_Abbreviated'


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassificationModel(
  (l1): LongformerForSequenceClassification(
    (longformer): LongformerModel(
      (embeddings): LongformerEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (position_embeddings): Embedding(4098, 768, padding_idx=1)
      )
      (encoder): LongformerEncoder(
        (layer): ModuleList(
          (0-11): 12 x LongformerLayer(
            (attention): LongformerAttention(
              (self): LongformerSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (query_global): Linear(in_features=768, out_features=768, bias=True)
                (key

In [9]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

tokenizer = model.tokenizer()
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (96986, 2)
TEST Dataset: (10805, 2)


In [10]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [11]:
loss_fn = loss(model, LOSS_TYPE, BALANCED, training_set, training_loader, device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {
    "Custom Classification Report": lambda y_true, y_pred: custom_classification_report(y_true, y_pred),
    "Optimization Metric": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=0)
}


In [12]:
model.model_name

'allenai_longformer-base-4096_Unfrozen[8, 9, 10, 11]_Abbreviated_BCE'

In [ ]:
from training import TrainingHandler
trainer = TrainingHandler(model, optimizer, loss_fn, device, metrics_dict, directory=MODEL_DIRECTORY, base_path=MODEL_NAME)
trainer.train(training_loader, testing_loader, save=True, epochs=EPOCHS)

In [14]:
from evaluation import EvaluationHandler
evaluator = EvaluationHandler(model, device, metrics_dict)
loss, metrics_results, thresholds = evaluator.evaluate(testing_loader, loss_fn, optimize=True)
thresholds

Evaluation - Batch: 338 [ 7098/10805], Time: 99s 190ms/step, Loss: 0.121098
Optimizing Thresholds
Evaluation Results:
Average Loss: 0.1444
Optimization Metric: 0.7137

Custom Classification Report:
  Deviation / Discrepancy - Procedural:  binary_accuracy: 0.7812, precision: 0.7754, recall: 0.8830, f1-score: 0.8257, support: 6343.0000
  Aircraft Equipment:                    binary_accuracy: 0.9149, precision: 0.8914, recall: 0.8980, f1-score: 0.8947, support: 4351.0000
  Conflict:                              binary_accuracy: 0.9558, precision: 0.8576, recall: 0.8941, f1-score: 0.8755, support: 1879.0000
  Inflight Event / Encounter:            binary_accuracy: 0.9003, precision: 0.7797, recall: 0.7742, f1-score: 0.7770, support: 2423.0000
  ATC Issue:                             binary_accuracy: 0.9150, precision: 0.8045, recall: 0.8402, f1-score: 0.8220, support: 2522.0000
  Deviation - Altitude:                  binary_accuracy: 0.9530, precision: 0.7360, recall: 0.7360, f1-score: 0

(0.14435112983341047,
 {'Custom Classification Report': {'Deviation / Discrepancy - Procedural': {'binary_accuracy': 0.7812124,
    'precision': 0.775439568046518,
    'recall': 0.8830206526880026,
    'f1-score': 0.8257408226448474,
    'support': 6343.0},
   'Aircraft Equipment': {'binary_accuracy': 0.9148542,
    'precision': 0.8913985854437599,
    'recall': 0.8979544932199495,
    'f1-score': 0.8946645294252347,
    'support': 4351.0},
   'Conflict': {'binary_accuracy': 0.9557612,
    'precision': 0.8575803981623277,
    'recall': 0.8940926024481107,
    'f1-score': 0.8754559666492965,
    'support': 1879.0},
   'Inflight Event / Encounter': {'binary_accuracy': 0.9003239,
    'precision': 0.7797173732335827,
    'recall': 0.7742468014857614,
    'f1-score': 0.7769724580658521,
    'support': 2423.0},
   'ATC Issue': {'binary_accuracy': 0.91503936,
    'precision': 0.8044798785117692,
    'recall': 0.8402061855670103,
    'f1-score': 0.821955003878976,
    'support': 2522.0},
   'D

In [15]:
from model_management import ModelManager
ModelManager.save_results(model.model_name, metrics_results, thresholds, "results.csv")